# 🔧 Prétraitement des Données

Nettoyage et transformation des événements pour créer des documents optimisés pour le RAG.

**Étapes :**

1. Chargement des données validées
2. Nettoyage du HTML et des textes
3. Extraction des informations clés
4. Création de documents structurés
5. Export pour embeddings


## 1️⃣ Configuration


In [1]:
import json
import re
from pathlib import Path
from datetime import datetime
from typing import List, Dict

import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

from src.config.constants import PROCESSED_DATA_DIR, PROCESSED_EVENTS_FILE

print("✅ Configuration OK")

✅ Configuration OK


## 2️⃣ Chargement des Données


In [ ]:
# Charger les données validées du notebook 01
processed_path = PROCESSED_DATA_DIR / PROCESSED_EVENTS_FILE

with open(processed_path, "r", encoding="utf-8") as f:
    events = json.load(f)

print(f"📊 {len(events)} événements chargés")
print(f"\n📋 Exemple d'événement :")
print(json.dumps(events[0], indent=2, ensure_ascii=False)[:500])

📊 497 événements chargés

📋 Exemple d'événement :
{
  "uid": "41320176",
  "title": "Formation Civique et Citoyenne \"Cadre de vie et architecture\"",
  "description": "Formation obligatoire à l'attention des volontaires de Service Civique, proposée par la MAV PACA.",
  "start_date": "2022-06-14T07:00:00+00:00",
  "end_date": "2025-12-31T22:59:00+00:00",
  "city": "Marseille",
  "address": "12 Bd Théodore Thurner 13006 Marseille",
  "url": "https://openagenda.com/culture/events/formation-civique-et-citoyenne-cadre-de-vie-et-architecture-en-pres


## 3️⃣ Fonctions de Nettoyage


In [ ]:
def clean_html(text: str) -> str:
    """Nettoie le HTML et retourne du texte brut."""
    if not text:
        return ""

    # Parser HTML
    soup = BeautifulSoup(text, "html.parser")

    # Extraire le texte
    text = soup.get_text(separator=" ", strip=True)

    # Nettoyer espaces multiples
    text = re.sub(r"\s+", " ", text)

    return text.strip()


def format_date(date_str: str) -> str:
    """Formate une date ISO en format lisible."""
    try:
        dt = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
        return dt.strftime("%d/%m/%Y à %H:%M")
    except:
        return date_str


def truncate_text(text: str, max_length: int = 500) -> str:
    """Tronque le texte tout en gardant les phrases complètes."""
    if len(text) <= max_length:
        return text

    # Trouver la dernière phrase complète
    truncated = text[:max_length]
    last_period = truncated.rfind(".")

    if last_period > 0:
        return truncated[: last_period + 1]

    return truncated + "..."


print("✅ Fonctions de nettoyage définies")

✅ Fonctions de nettoyage définies


## 4️⃣ Création des Documents RAG


In [ ]:
def create_rag_document(event: Dict) -> Dict:
    """Crée un document structuré pour le RAG."""

    # Nettoyer la description
    description = clean_html(event.get("description", ""))

    # Formater les dates
    start = format_date(event.get("start_date", ""))
    end = format_date(event.get("end_date", ""))

    # Créer le texte principal pour l'embedding
    content_parts = [
        f"Titre: {event.get('title', 'Sans titre')}",
        f"Ville: {event.get('city', 'Non spécifié')}",
        f"Date: Du {start} au {end}",
    ]

    if description:
        content_parts.append(f"Description: {truncate_text(description)}")

    if event.get("address"):
        content_parts.append(f"Adresse: {event['address']}")

    # Document final
    return {
        "id": event["uid"],
        "title": event.get("title", ""),
        "content": "\n".join(content_parts),
        "metadata": {
            "uid": event["uid"],
            "city": event.get("city", ""),
            "start_date": event.get("start_date", ""),
            "end_date": event.get("end_date", ""),
            "url": event.get("url", ""),
            "address": event.get("address", ""),
        },
    }


print("✅ Fonction de création de documents définie")

✅ Fonction de création de documents définie


In [5]:
# Créer les documents RAG
rag_documents = []

for event in tqdm(events, desc="🔧 Prétraitement"):
    doc = create_rag_document(event)
    rag_documents.append(doc)

print(f"\n✅ {len(rag_documents)} documents créés")
print(f"\n📋 Exemple de document RAG :")
print(json.dumps(rag_documents[0], indent=2, ensure_ascii=False))

🔧 Prétraitement:   0%|          | 0/497 [00:00<?, ?it/s]


✅ 497 documents créés

📋 Exemple de document RAG :
{
  "id": "41320176",
  "title": "Formation Civique et Citoyenne \"Cadre de vie et architecture\"",
  "content": "Titre: Formation Civique et Citoyenne \"Cadre de vie et architecture\"\nVille: Marseille\nDate: Du 14/06/2022 à 07:00 au 31/12/2025 à 22:59\nDescription: Formation obligatoire à l'attention des volontaires de Service Civique, proposée par la MAV PACA.\nAdresse: 12 Bd Théodore Thurner 13006 Marseille",
  "metadata": {
    "uid": "41320176",
    "city": "Marseille",
    "start_date": "2022-06-14T07:00:00+00:00",
    "end_date": "2025-12-31T22:59:00+00:00",
    "url": "https://openagenda.com/culture/events/formation-civique-et-citoyenne-cadre-de-vie-et-architecture-en-presentiel",
    "address": "12 Bd Théodore Thurner 13006 Marseille"
  }
}


## 5️⃣ Statistiques


In [ ]:
# Analyser les longueurs de contenu
content_lengths = [len(doc["content"]) for doc in rag_documents]

print("📊 Statistiques des documents :")
print(f"  Longueur moyenne : {sum(content_lengths) / len(content_lengths):.0f} caractères")
print(f"  Longueur min : {min(content_lengths)} caractères")
print(f"  Longueur max : {max(content_lengths)} caractères")

# Vérifier les métadonnées
cities = [doc["metadata"]["city"] for doc in rag_documents if doc["metadata"]["city"]]
print(f"\n📍 {len(set(cities))} villes différentes")
print(f"\n🏙️ Top 5 villes :")
from collections import Counter

for city, count in Counter(cities).most_common(5):
    print(f"  • {city}: {count} événements")

📊 Statistiques des documents :
  Longueur moyenne : 317 caractères
  Longueur min : 157 caractères
  Longueur max : 488 caractères

📍 1 villes différentes

🏙️ Top 5 villes :
  • Marseille: 497 événements


## 6️⃣ Export


In [ ]:
# Export JSON
output_path = PROCESSED_DATA_DIR / "rag_documents.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(rag_documents, f, ensure_ascii=False, indent=2)

print(f"💾 Documents exportés : {output_path}")
print(f"\n✅ Prétraitement terminé !")
print(f"\n➡️ Prochaine étape : Notebook 03 - Création des embeddings")

💾 Documents exportés : /Users/ppluton/Documents/Repositories/OC7---Projet-RAG-Assistant-Intelligent-Events/data/processed/rag_documents.json

✅ Prétraitement terminé !

➡️ Prochaine étape : Notebook 03 - Création des embeddings
